Name: Vansh Kolte

Roll no: 62

Batch: C4

In [1]:
class CodeGenerator:
    def __init__(self, tac):
        self.tac = tac 
        self.registers = []  
        self.instructions = [] 
        self.reg_map = {}  
        self.memory = {}  
        self.next_memory_location = 100  

    
    def initialize_registers(self, num_registers=4):
       
        self.registers = [f"R{i+1}" for i in range(num_registers)]

  
    def allocate_register(self, var):
        if var in self.reg_map:
            return self.reg_map[var]  
        if self.registers:
            reg = self.registers.pop(0)  
            self.reg_map[var] = reg
            return reg
        else:
           
            self.spill_register()
          
            return self.allocate_register(var)

    def free_register(self, reg):
        self.registers.append(reg)

    def spill_register(self):
        if not self.reg_map:
            raise Exception("No registers to spill")

       
        var_to_spill = next(iter(self.reg_map))
        spilled_reg = self.reg_map.pop(var_to_spill)
        
        
        memory_location = self.next_memory_location
        self.memory[var_to_spill] = memory_location
        self.next_memory_location += 4  
        
      
        self.instructions.append(f"STORE {spilled_reg}, [{memory_location}]")
        self.free_register(spilled_reg)

    def load_from_memory(self, var):
        if var in self.memory:
            
            reg = self.allocate_register(var)
            memory_location = self.memory[var]
            self.instructions.append(f"LOAD {reg}, [{memory_location}]")
            return reg
        else:
            raise Exception(f"Variable {var} not found in memory")

    def generate_code_for_line(self, line):
        parts = line.split()
        if "=" in parts:
            
            dest = parts[0]
            if len(parts) == 3:  
                src = parts[2]
                if src.isdigit(): 
                    reg = self.allocate_register(dest)
                    self.instructions.append(f"MOV {reg}, #{src}")  
                else:
                    reg_src = self.allocate_register(src) if src not in self.memory else self.load_from_memory(src)
                    reg_dest = self.allocate_register(dest)
                    self.instructions.append(f"MOV {reg_dest}, {reg_src}")  
            elif len(parts) == 5:  
                op1, operator, op2 = parts[2], parts[3], parts[4]
                reg_op1 = self.allocate_register(op1) if op1 not in self.memory else self.load_from_memory(op1)
                reg_op2 = self.allocate_register(op2) if op2 not in self.memory else self.load_from_memory(op2)
                reg_dest = self.allocate_register(dest)

                if operator == '+':
                    self.instructions.append(f"ADD {reg_dest}, {reg_op1}, {reg_op2}")
                elif operator == '-':
                    self.instructions.append(f"SUB {reg_dest}, {reg_op1}, {reg_op2}")
                elif operator == '*':
                    self.instructions.append(f"MUL {reg_dest}, {reg_op1}, {reg_op2}")
                elif operator == '/':
                    self.instructions.append(f"DIV {reg_dest}, {reg_op1}, {reg_op2}")

    def generate_code(self):
        for line in self.tac:
            self.generate_code_for_line(line)

    def get_generated_code(self):
        return self.instructions


if __name__ == "__main__":
    
    tac = [
        "t1 = 4 + 5",
        "t2 = t1",
        "t3 = t2 + 3",
        "t4 = t1 + t3",
        "t5 = t2 * t4"
    ]

    codegen = CodeGenerator(tac)
    codegen.initialize_registers(num_registers=4)  
    codegen.generate_code()

    generated_code = codegen.get_generated_code()
    print("Generated Assembly-like Code:")
    for line in generated_code:
        print(line)

Generated Assembly-like Code:
ADD R3, R1, R2
MOV R4, R3
STORE R1, [100]
STORE R2, [104]
ADD R2, R4, R1
STORE R3, [108]
ADD R3, R3, R2
STORE R4, [112]
MUL R4, R4, R3
